In [42]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path 

import os

import torch
import torch.optim as optim

import random 

# fastai
from fastai import *
from fastai.text import *
from fastai.callbacks import *

# transformers
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

ModuleNotFoundError: No module named 'fastai.callbacks'

In [44]:
!pip install fastai==2.0.15
!pip install fastai2==0.0.30
!pip install fastcore==1.0.16

In [7]:
train = pd.read_csv("/Users/olayile/GBV_tweet_analysis/gender-based-violence-tweet-classification/Train.csv")

In [10]:
test= pd.read_csv("/Users/olayile/GBV_tweet_analysis/gender-based-violence-tweet-classification/Test.csv")

In [8]:
train

,Tweet_ID,tweet,type
0,ID_0022DWKP,Had a dream i got raped last night. By a guy i...,sexual_violence
1,ID_00395QYM,he thought the word raped means sex and told m...,sexual_violence
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,ID_004F7516,Chessy Prout can do better by telling the trut...,sexual_violence
...,...,...,...
39645,ID_ZZTLP2L5,"ENTRY 1299: 21F. 23M, BF’s cousin. Got drunk o...",sexual_violence
39646,ID_ZZXB10M6,So you’re telling me Emmanuel Macron was groom...,sexual_violence
39647,ID_ZZY8B7FJ,"My wife regularly beats me, I get dirty slaps ...",Physical_violence
39648,ID_ZZYAFD8L,Me: Hey babe! Police officer boyfriend: is tha...,sexual_violence


In [9]:

train.tweet = train.tweet.str.replace(r'http(\S)+', r'')
train.tweet = train.tweet.str.replace(r'http ...', r'')
train.tweet = train.tweet.str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
train.tweet = train.tweet.str.replace(r'@[\S]+',r'')

# Remove non-ascii words or characters
#train.tweet = [''.join([i if ord(i) < 128 else '' for i in tweet]) for tweet in train.tweet]
train.tweet = train.tweet.str.replace(r'_[\S]?',r'')

# Remove extra space
train.tweet = train.tweet.str.replace(r'[ ]{2, }',r' ')

# Remove &, < and >
train.tweet = train.tweet.str.replace(r'&amp;?',r'and')
train.tweet = train.tweet.str.replace(r'&lt;',r'<')
train.tweet = train.tweet.str.replace(r'&gt;',r'>')

# Insert space between words and punctuation marks
train.tweet = train.tweet.str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
train.tweet = train.tweet.str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')

# Lowercased and strip
#train.tweet = train.tweet.str.lower()
train.tweet = train.tweet.str.strip()

/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_35427/1704451603.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train.tweet = train.tweet.str.replace(r'http(\S)+', r'')
/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_35427/1704451603.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train.tweet = train.tweet.str.replace(r'http ...', r'')
/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_35427/1704451603.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train.tweet = train.tweet.str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_35427/1704451603.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train.tweet = train.tweet.str.replace(r'@[\S]+',r'')
/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipyke

In [11]:
test.tweet = test.tweet.str.replace(r'http(\S)+', r'')
test.tweet = test.tweet.str.replace(r'http ...', r'')
test.tweet = test.tweet.str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
test.tweet = test.tweet.str.replace(r'@[\S]+',r'')

# Remove non-ascii words or characters
#test.tweet = [''.join([i if ord(i) < 128 else '' for i in tweet]) for tweet in test.tweet]
test.tweet = test.tweet.str.replace(r'_[\S]?',r'')

# Remove extra space
test.tweet = test.tweet.str.replace(r'[ ]{2, }',r' ')

# Remove &, < and >
test.tweet = test.tweet.str.replace(r'&amp;?',r'and')
test.tweet = test.tweet.str.replace(r'&lt;',r'<')
test.tweet = test.tweet.str.replace(r'&gt;',r'>')

# Insert space between words and punctuation marks
test.tweet = test.tweet.str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
test.tweet = test.tweet.str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')

# Lowercased and strip
#test.tweet = test.tweet.str.lower()
test.tweet = test.tweet.str.strip()

/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_35427/3768140800.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test.tweet = test.tweet.str.replace(r'http(\S)+', r'')
/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_35427/3768140800.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test.tweet = test.tweet.str.replace(r'http ...', r'')
/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_35427/3768140800.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test.tweet = test.tweet.str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_35427/3768140800.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test.tweet = test.tweet.str.replace(r'@[\S]+',r'')
/var/folders/gg/6z7ytz0d19j5jqwl4jxzqkw00000gn/T/ipykernel_354

In [12]:
train

,Tweet_ID,tweet,type
0,ID_0022DWKP,Had a dream i got raped last night . By a guy ...,sexual_violence
1,ID_00395QYM,he thought the word raped means sex and told m...,sexual_violence
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,sexual_violence
4,ID_004F7516,Chessy Prout can do better by telling the trut...,sexual_violence
...,...,...,...
39645,ID_ZZTLP2L5,"ENTRY 1299 : 21F . 23M , BF ’ s cousin . Got d...",sexual_violence
39646,ID_ZZXB10M6,So you ’ re telling me Emmanuel Macron was gro...,sexual_violence
39647,ID_ZZY8B7FJ,"My wife regularly beats me , I get dirty slaps...",Physical_violence
39648,ID_ZZYAFD8L,Me : Hey babe ! Police officer boyfriend : is ...,sexual_violence


In [13]:
train['text_length'] = [len(tweet.split(' ')) for tweet in train.tweet]

In [15]:
train = train[train['text_length']>0]

In [16]:
train

,Tweet_ID,tweet,type,text_length
0,ID_0022DWKP,Had a dream i got raped last night . By a guy ...,sexual_violence,50
1,ID_00395QYM,he thought the word raped means sex and told m...,sexual_violence,23
2,ID_003EOSSF,She NOT TALKING TO ME I WAS RAPED BY 2 MEN 1 M...,sexual_violence,24
3,ID_004BBHOD,I was sexually abused for 3 years at age 4 to ...,sexual_violence,75
4,ID_004F7516,Chessy Prout can do better by telling the trut...,sexual_violence,62
...,...,...,...,...
39645,ID_ZZTLP2L5,"ENTRY 1299 : 21F . 23M , BF ’ s cousin . Got d...",sexual_violence,69
39646,ID_ZZXB10M6,So you ’ re telling me Emmanuel Macron was gro...,sexual_violence,46
39647,ID_ZZY8B7FJ,"My wife regularly beats me , I get dirty slaps...",Physical_violence,15
39648,ID_ZZYAFD8L,Me : Hey babe ! Police officer boyfriend : is ...,sexual_violence,36


In [17]:
MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)
}

In [18]:
# Parameters
seed = 15
use_fp16 = False
bs = 32

model_type = 'roberta'
pretrained_model_name = 'roberta-base'

In [19]:
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]

In [23]:
from transformers import BERT_PRETRAINED_CONFIG_ARCHIVE_MAP
BERT_PRETRAINED_CONFIG_ARCHIVE_MAP.keys()

dict_keys(['bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-base-cased-finetuned-mrpc', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased'])

In [24]:

def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) 
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [25]:
seed_all(seed)

In [30]:
 from fastai.text.all import *
 from typing import List

class TransformersBaseTokenizer(BaseTokenizer):
    def __init__(self, pretrained_tokenizer: PreTrainedTokenizer, model_type = 'bert', **kwargs):
        self._pretrained_tokenizer = pretrained_tokenizer
        self.max_seq_len = pretrained_tokenizer.model_max_length
        self.model_type = model_type

    def __call__(self, *args, **kwargs): 
        return self

    def tokenizer(self, t:str) -> List[str]:
        CLS = self._pretrained_tokenizer.cls_token
        SEP = self._pretrained_tokenizer.sep_token
        if self.model_type in ['roberta']:
            tokens = self._pretrained_tokenizer.tokenize(t, add_prefix_space=True)[:self.max_seq_len - 2]
            tokens = [CLS] + tokens + [SEP]
        else:
            tokens = self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2]
            if self.model_type in ['xlnet']:
                tokens = tokens + [SEP] +  [CLS]
            else:
                tokens = [CLS] + tokens + [SEP]
        return tokens

In [38]:

transformer_tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)
transformer_base_tokenizer = TransformersBaseTokenizer(pretrained_tokenizer = transformer_tokenizer, model_type = model_type)
fastai_tokenizer = Tokenizer(tok = transformer_base_tokenizer)

In [40]:
class TransformersVocab(Vocab):
    def __init__(self, tokenizer: PreTrainedTokenizer):
        super(TransformersVocab, self).__init__(itos = [])
        self.tokenizer = tokenizer
    
    def numericalize(self, t:Collection[str]) -> List[int]:
        return self.tokenizer.convert_tokens_to_ids(t)
        #return self.tokenizer.encode(t)

    def textify(self, nums:Collection[int], sep=' ') -> List[str]:
        nums = np.array(nums).tolist()
        return sep.join(self.tokenizer.convert_ids_to_tokens(nums)) if sep is not None else self.tokenizer.convert_ids_to_tokens(nums)
    
    def __getstate__(self):
        return {'itos':self.itos, 'tokenizer':self.tokenizer}

    def __setstate__(self, state:dict):
        self.itos = state['itos']
        self.tokenizer = state['tokenizer']
        self.stoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.itos)})

NameError: name 'Vocab' is not defined